<a href="https://colab.research.google.com/github/Prezii/Project/blob/main/DATA_DOWNLOAD_Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':"1zqCgJ5SZCFlH8BEY3JiQjM2pv7SSTLmC"})   # replace the id with id of file you want to access
downloaded.GetContentFile('tremor_events-2011-10-01T00_00_00-2015-02-28T23_59_59.csv')        # replace the file name with your file

In [4]:
import sys
#Install obspy
!{sys.executable} -m pip install obspy
import obspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.3 MB 6.3 MB/s 


In [5]:
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install geos pyproj pyshp
!{sys.executable} -m pip install https://downloads.sourceforge.net/project/matplotlib/matplotlib-toolkits/basemap-1.1.0/basemap-1.1.0.tar.gz
!{sys.executable} -m pip install instaseis
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install cartopy
!{sys.executable} -m pip install mpi4py #Run in parallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400 kB 4.3 MB/s 
     |████████████████████████████████| 6.3 MB 45.9 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  ERROR: HTTP error 404 while getting https://downloads.sourceforge.net/project/matplotlib/matplotlib-toolkits/basemap-1.1.0/basemap-1.1.0.tar.gz
ERROR: Could not install requirement https://downloads.sourceforge.net/project/matplotlib/matplotlib-toolkits/basemap-1.1.0/basemap-1.1.0.tar.gz because of HTTP error 404 Client Error: Not Found for url: https://downloads.sourceforge.net/project/matplotlib/matplotlib-toolkits/basemap-1.1.0/basemap-1.1.0.tar.gz for URL https://downloads.sourceforge.net/project/matplotlib/matplotlib-to

In [6]:
import os
import sys

In [7]:
os.getcwd()
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [8]:
from obspy.clients.syngine import Client as SyngineClient

In [10]:
import numpy as np

#Experiment CVS

zeros = np.zeros(5)

In [11]:
np.savetxt("save.csv", zeros, delimiter=",")



```
def InitProjectFolderTree(directory,projectName,init=False):
    
    '''
    Function that creates init folder tree for new project, if init=True, all content under directory
    +projectName is deleted and a new tree is created. The project folder tree is created to use MudPy,
    all the folders where Mudpy stores information are created.
    '''
    
    if(init==True):
        !rm -r {directory}{projectName}
        with open(str(directory)+'slipInversion.inv.py','r') as file:
            data=file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='home'):
                data[i]='home='+'"'+str(directory)+'"'+'\n'
            if(line[:12]=='project_name'):
                data[i]='project_name='+'"'+str(projectName)+'"'+'\n'
            if(line[:4]=='init'):
                data[i]='init=1 #Initalize project\n'
            if(line[:10]=='make_green'):
                data[i]='make_green=0 #Compute GFs\n'
            if(line[:15]=='make_synthetics'):
                data[i]='make_synthetics=0 #Compute synthetics for a given model at given stations\n'
            if(line[:6]=='invert'):
                data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
            
        !python {directory}slipInversion.inv.py
        
        with open(str(directory)+'slipInversion.inv.py', 'r') as file:
            data = file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='init'):
                data[i]='init=0 #Initalize project\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
    #!nautilus {directory}{projectName}
    !open {directory}{projectName}
    
```



In [9]:
#Choose a home Directory for project (everything will be saved in this directory)
directory='/Users/jeanbaptistetary/Documents/SSI/'

#Name of the folder that will contain all the Near Field / Regional results.
projectName='NearField'

#Select init=True to initialize project folder tree, If the tree already exists select False 
#WARNING: IF TRUE THE EXISTING FOLDER "projectName" WILL BE DELETED!!!
InitProjectFolderTree(directory,projectName,init=False)

NameError: ignored



```
def greenFunctionList(stream,directory,projectName):
    '''
    Function that modifies the green function list that tells the main
    program where the waveforms are located and the location of each
    station.
    directory=current working space.
    projectName=NearField/FarField/JointInversion.
    inv=it needs an inventory that tells the location of the station. For 
    this just run the function createInventory().
    '''
    file = open(str(directory)+str(projectName)+'/data/station_info/mutata.gflist', 'wb')
    file.write('#station lon lat "static,disp,vel,tsun,insar" "Static file, displacement file, velocity file,tsunami file,insar file" "static sigmas(n,e,u),displacement sigmas(n,e,u),velocity sigmas(n,e,u),tsunami sigma,tsunami sigma"\n')
    code1='null'
    
    for tr in stream:
        code=repr(tr.stats.station)[2:-1]
        longitude=repr(tr.stats.longitude)
        latitude=repr(tr.stats.latitude)
        if len(code)>4:
            code=code[:4]
        if len(code)<4:
            code=code+" "
        if float(longitude)>0:
            longitude=" "+longitude
        if len(longitude)>7:
            longitude=longitude[:7]
        if len(longitude)<7:
            longitude=longitude+"0"
        if float(latitude)>0:
            latitude=" "+latitude
        if len(latitude)>7:
            latitude=latitude[:7]
        if len(latitude)<7:
            latitude=latitude+"0"
        if code1!=code:
            code1=code
            file.write((code) + " " +longitude+ " "+ latitude + " "+ "0	1	0	0	0	/foo/bar	"+str(directory)+str(projectName)+"/data/waveforms/"+tr.stats.station+".disp	/foo/bar	/foo/bar	/foo/bar	1	1	1	1.0 1.0 1.0	1	1	1	0	0	0	0	1	0\n")
    file.close()
    
    !open {directory}{projectName}/data/station_info/mutata.gflist -a textedit
```





```
def InitProjectFolderTree(directory,projectName,init=False):
    
    '''
    Function that creates init folder tree for new project, if init=True, all content under directory
    +projectName is deleted and a new tree is created. The project folder tree is created to use MudPy,
    all the folders where Mudpy stores information are created.
    '''
    
    if(init==True):
        !rm -r {directory}{projectName}
        with open(str(directory)+'slipInversion.inv.py','r') as file:
            data=file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='home'):
                data[i]='home='+'"'+str(directory)+'"'+'\n'
            if(line[:12]=='project_name'):
                data[i]='project_name='+'"'+str(projectName)+'"'+'\n'
            if(line[:4]=='init'):
                data[i]='init=1 #Initalize project\n'
            if(line[:10]=='make_green'):
                data[i]='make_green=0 #Compute GFs\n'
            if(line[:15]=='make_synthetics'):
                data[i]='make_synthetics=0 #Compute synthetics for a given model at given stations\n'
            if(line[:6]=='invert'):
                data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
            
        !python {directory}slipInversion.inv.py
        
        with open(str(directory)+'slipInversion.inv.py', 'r') as file:
            data = file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='init'):
                data[i]='init=0 #Initalize project\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
    #!nautilus {directory}{projectName}
    !open {directory}{projectName}
```



##!IMPORTANT!

In [ ]:
###placeObservedTraces, IMPORTANT FUNCTION



```
#######################################Project Functions##############################################

def InitProjectFolderTree(directory,projectName,init=False):
    
    '''
    Function that creates init folder tree for new project, if init=True, all content under directory
    +projectName is deleted and a new tree is created. The project folder tree is created to use MudPy,
    all the folders where Mudpy stores information are created.
    '''
    
    if(init==True):
        !rm -r {directory}{projectName}
        with open(str(directory)+'slipInversion.inv.py','r') as file:
            data=file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='home'):
                data[i]='home='+'"'+str(directory)+'"'+'\n'
            if(line[:12]=='project_name'):
                data[i]='project_name='+'"'+str(projectName)+'"'+'\n'
            if(line[:4]=='init'):
                data[i]='init=1 #Initalize project\n'
            if(line[:10]=='make_green'):
                data[i]='make_green=0 #Compute GFs\n'
            if(line[:15]=='make_synthetics'):
                data[i]='make_synthetics=0 #Compute synthetics for a given model at given stations\n'
            if(line[:6]=='invert'):
                data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
            
        !python {directory}slipInversion.inv.py
        
        with open(str(directory)+'slipInversion.inv.py', 'r') as file:
            data = file.readlines()
        for i,line in enumerate(data):
            if(line[:4]=='init'):
                data[i]='init=0 #Initalize project\n'
        with open(str(directory)+'slipInversion.inv.py', 'w') as file:
            file.writelines(data)
    #!nautilus {directory}{projectName}
    !open {directory}{projectName}
    
def newIteration(directory,projectName,dontResetFaultParameters=False):
    '''
    This function starts a new iteration erasing all the data, be careful.
    '''
    !rm {directory}{projectName}/data/waveforms/*
    !rm {directory}{projectName}/data/station_info/mutata.gflist
    if dontResetFaultParameters==False:
        !rm {directory}{projectName}/data/model_info/mutata.fault
    !rm {directory}{projectName}/output/inverse_models/models/*
    !rm {directory}{projectName}/output/inverse_models/waveforms/*
    !rm -r {directory}{projectName}/GFs/dynamic/* 
    
def placeObservedTraces(directory,projectName,stream):
    '''
    Function that generates the sac files and place them in the right folder.
    directory=current working space.
    projectName=one of the following: NearField/FarField/JointInversion.
    '''
    for tr in stream:
        if tr.stats.channel=='BHN' or tr.stats.channel=='HHN':
            comp='n'
        if tr.stats.channel=='BHE' or tr.stats.channel=='HHE':
            comp='e'
        if tr.stats.channel=='BHZ' or tr.stats.channel=='HHZ':
            comp='u'
        tr.write(str(directory)+str(projectName)+'/data/waveforms/'+str(tr.stats.station)+'.disp.'+comp, format='SAC')
    #!nautilus {directory}{projectName}/data/waveforms/
    !open {directory}{projectName}/data/waveforms/
    
def removeTrace(traces,listOfTraces):
    '''
    Removes Trace
    '''
    for tr in traces:
        for t in listOfTraces:
            if (tr.stats.station==t):
                traces.remove(tr)
    return traces

def removeTraceLoc(traces,staname,loc):
    '''
    Removes Traces for a specific station and location (when a station have more than one)
    '''
    for tr in traces.select(station=staname, location=loc):
        traces.remove(tr)
    
    return traces

def changeProject(projectName,directory):
    '''
    Function that changes the working folder in the main file.
    directory=current working space.
    projectName=one of the following: NearField/FarField/JointInversion.
    '''
    with open(str(directory)+'slipInversion.inv.py', 'r') as file:
        data = file.readlines()
    for i, line in enumerate(data):
        if line[:12]=='project_name':
            data[i]="project_name='"+str(projectName)+"'\n"
    with open(str(directory)+'slipInversion.inv.py', 'w') as file:
        file.writelines( data )
    #!open {directory}slipInversion.inv.py -a Xcode
    !gedit {directory}slipInversion.inv.py

def createInventory(stream,starttime,endtime):
    '''
    This function generates an inventory with the info from all the stations.
    stream=contains all the traces in a stream using the obspy function Stream()
    starttime=starttime of the trace record.
    endtime=endtime of the trace record.
    '''
    ###Get the information for all the stations.
    from obspy.clients.fdsn import Client
    client = Client("IRIS")
    first=True
    for tr in stream:
        if (first==True):
            inv = client.get_stations(network=tr.stats.network, station=tr.stats.station, 
                                      location='00', channel='BHZ',starttime=starttime,
                                      endtime=endtime, level="response")
            first=False
        else:
            inv += client.get_stations(network=tr.stats.network, station=tr.stats.station, 
                                       location='00', channel='BHZ',starttime=starttime,
                                       endtime=endtime, level="response")
    
    return (inv)

def prepareRunFile(nl,nw,event_longitude,event_latitude,event_depth):
    '''
    Run File is prepared with number of subfaults across length nl and across width nw. The run file is opened
    for the user to check if everything is in its place. Check and custom reg parameters, filters, project name
    and homedir. Check and custom all parameters and check that the input file names are correct.
    '''
    with open(str(directory)+'slipInversion.inv.py', 'r') as file:
        # read a list of lines into data
        data = file.readlines()
    for i, line in enumerate(data):
        if line[:7]=='nstrike':
            data[i]='nstrike='+repr(nl)+' ; ndip='+repr(nw)+' ; nfaults=(nstrike,ndip) #set nstrike to total no. of faults and ndip to 1 if using Tikh\n'
        if line[:9]=='epicenter':
            data[i]=('epicenter=np.array([ '+repr(event_longitude) +' , '+repr(event_latitude)+' ,'
                  +repr(event_depth)+'])\n')
            print(data[i])
    with open(str(directory)+'slipInversion.inv.py', 'w') as file:
        file.writelines( data )
    #!open {directory}SlipInversion.inv.py -a Xcode
    !open {directory}slipInversion.inv.py -a textedit
    
def runInversion(directory,projectName,makeGreen=False,makeSynthetics=False,invert=False):
    '''
    Function that modifies the main file that controls the inversion so that it generates the 
    green functions, the synthetics and run the inversion.
    directory=current working space.
    projectName=one of the following: NearField/FarField/JointInversion.
    '''
    with open(str(directory)+'slipInversion.inv.py', 'r') as file:
        data = file.readlines()
    for i, line in enumerate(data):
        if line[:10]=='make_green':
            if makeGreen==True:
                data[i]='make_green=1 #Compute GFs\n'
            else:
                data[i]='make_green=0 #Do not compute GFs\n'
        if line[:15]=='make_synthetics':
            if makeSynthetics==True:
                data[i]='make_synthetics=1 #Compute synthetics for a given model at given stations\n'
            else:
                data[i]='make_synthetics=0 #Do not compute synthetics for a given model at given stations\n'
        if line[:6]=='invert':
            if invert==True:
                data[i]='invert=1  # =1 runs inversion, =0 does nothing\n'
            else:
                data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
        if line[:12]=='project_name':
            data[i]="project_name='"+str(projectName)+"'\n"
    with open(str(directory)+'slipInversion.inv.py', 'w') as file:
        file.writelines( data )
    print('This might take some time, go grab something to eat ....')
        
    !python {directory}slipInversion.inv.py

    '''
    Modifies the main file that controls the inversion so that it no longer perform any task.
    '''
    with open(str(directory)+'slipInversion.inv.py', 'r') as file:
        # read a list of lines into data
        data = file.readlines()
    for i, line in enumerate(data):
        if line[:10]=='make_green':
            data[i]='make_green=0 #Do not compute GFs\n'
        if line[:15]=='make_synthetics':
            data[i]='make_synthetics=0 #Do not compute synthetics for a given model at given stations\n'
        if line[:6]=='invert':
            data[i]='invert=0  # =1 runs inversion, =0 does nothing\n'
    with open(str(directory)+'slipInversion.inv.py', 'w') as file:
        file.writelines( data )
    
###########################################Plotting Functions##############################################
```



In [ ]:
#/content/tremor_events-2011-10-01T00_00_00-2015-02-28T23_59_59.csv